In [ ]:
# Equivalent script in a Python Notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Simulation parameters
L = 50.0
NX = 1024
dx = L / NX
x = np.linspace(0, L, NX)
T = 20.0
dt = 0.01
nstep = int(T / dt)
skip = 20  # plot snapshot every `skip` steps

# Physical constants (set to 1 for atomic units)
hbar = 1.0
m = 1.0

# Initial wave packet
x0 = 12.0
sigma = 1.0
k0 = 3.0
psi = np.exp(-(x - x0)**2 / (2 * sigma**2)) * np.exp(1j * k0 * x)
psi /= np.sqrt(np.sum(np.abs(psi)**2) * dx)

# Potential: box + central barrier
V = np.zeros_like(x)
V[np.logical_or(x <= L/50, x >= L - L/50)] = 100.0  # high edge walls
V[np.logical_and(x > (L - 2.0) / 2, x < (L + 2.0) / 2)] = 4.0  # central barrier

# FFT momentum space
p = np.fft.fftshift(np.fft.fftfreq(NX, d=dx) * 2 * np.pi)

# Time evolution operators
expV = np.exp(-1j * V * dt / (2 * hbar))
expT = np.exp(-1j * (hbar * p)**2 / (2 * m) * dt / hbar)

# Plot setup
fig, ax = plt.subplots(figsize=(8, 4))
line_re, = ax.plot(x, np.real(psi), label='Re[ψ]', lw=0.5, color="red")
line_im, = ax.plot(x, np.imag(psi), label='Im[ψ]', lw=0.5, color="blue")
line_abs, = ax.plot(x, np.abs(psi)**2, label='|ψ|²', lw=0.5, color="black")
ax.plot(x, V / np.max(V) * np.max(np.abs(psi)**2), label='V(x)', lw=2, color="lime")
ax.set_ylim(-0.2, 1.2 * np.max(np.abs(psi)**2))
ax.set_xlim(0, L)
ax.legend(loc='upper right')

# Update function: simulate skip steps, plot 1 frame
def update(frame):
    global psi
    for _ in range(skip):
        psi = expV * psi
        psi = np.fft.fftshift(np.fft.fft(psi))
        psi = expT * psi
        psi = np.fft.ifft(np.fft.ifftshift(psi))
        psi = expV * psi
    line_re.set_ydata(np.real(psi))
    line_im.set_ydata(np.imag(psi))
    line_abs.set_ydata(np.abs(psi)**2)
    return line_re, line_im, line_abs

# Animate
plt.rcParams["animation.html"] = "jshtml"
ani = FuncAnimation(fig, update, frames=nstep//skip, blit=True, interval=30)
plt.close(fig) # avoid showing the final frame twice
HTML(ani.to_jshtml())